In [1]:
import os

# Hauptverzeichnis
base_dir = r"G:\43_DJI_Mini3Pro"

# Liste aller Ordner im Verzeichnis
print("Gefundene Ordner:")
for entry in sorted(os.listdir(base_dir)):
    full_path = os.path.join(base_dir, entry)
    if os.path.isdir(full_path):
        print(f"- {entry}")


Gefundene Ordner:
- 20250221_Mini3Pro_Ahorn
- 20250221_Mini3Pro_Kirsche
- 20250221_Mini3Pro_Linde
- 20250228_Mini3Pro_Ahorn
- 20250228_Mini3Pro_Kirsche
- 20250228_Mini3Pro_Linde
- 20250305_Mini3Pro_Ahorn
- 20250305_Mini3Pro_Kirsche
- 20250305_Mini3Pro_Linde
- 20250311_Mini3Pro_Kirsche
- 20250314_Mini3Pro_Ahorn
- 20250314_Mini3Pro_Kirsche
- 20250317_Mini3Pro_Ahorn
- 20250317_Mini3Pro_Kirsche
- 20250319_Mini3Pro_Kirsche
- 20250326_Mini3Pro_Ahorn_nichtKomplett
- 20250326_Mini3Pro_Kirsche
- 20250326_Mini3Pro_Kirsche_Detail
- 20250331_Mini3Pro_Kirsche
- 20250403_Mini3Pro_Ahorn
- 20250403_Mini3Pro_Linde
- 20250411_Mini3Pro_Kirsche
- 20250423_Mini3Pro_Kirsche
- 20250508_Mini3Pro_Ahorn
- 20250508_Mini3Pro_Linde


In [6]:
import os
import re

# Hauptverzeichnis
base_dir = r"G:\43_DJI_Mini3Pro"

# Regulärer Ausdruck für exakte Struktur: 8-stelliges Datum + "_Mini3Pro_Kirsche"
pattern = re.compile(r"^\d{8}_Mini3Pro_Linde$")

# Passende Ordner auflisten
print("Gefundene Kirsche-Ordner mit gültiger Struktur:")
for entry in sorted(os.listdir(base_dir)):
    full_path = os.path.join(base_dir, entry)
    if os.path.isdir(full_path) and pattern.match(entry):
        print(f"- {entry}")


Gefundene Kirsche-Ordner mit gültiger Struktur:
- 20250221_Mini3Pro_Linde
- 20250228_Mini3Pro_Linde
- 20250305_Mini3Pro_Linde
- 20250403_Mini3Pro_Linde
- 20250508_Mini3Pro_Linde


In [7]:
import os
import re

# Hauptverzeichnis
base_dir = r"G:\43_DJI_Mini3Pro"

# Gültige Bild-Endungen
valid_extensions = (".jpg", ".jpeg", ".tif", ".tiff", ".png")

# Ordner-Muster: exakter Match auf z. B. 20250314_Mini3Pro_Kirsche
pattern = re.compile(r"^\d{8}_Mini3Pro_Linde$")

print("Suche nach Bilddateien in gültigen Ordnern:\n")

for folder in sorted(os.listdir(base_dir)):
    if not pattern.match(folder):
        continue

    folder_path = os.path.join(base_dir, folder)
    if not os.path.isdir(folder_path):
        continue

    # Bilddateien suchen
    image_files = sorted([
        f for f in os.listdir(folder_path)
        if f.lower().endswith(valid_extensions)
    ])

    print(f"Ordner: {folder} ({len(image_files)} Bilddateien)")

    if not image_files:
        print("  (keine Bilddateien gefunden)\n")
        continue

    # Erste zwei und letzte zwei anzeigen
    preview_files = image_files[:2] + image_files[-2:] if len(image_files) > 4 else image_files

    for img in preview_files:
        print(f"  - {img}")
    print()


Suche nach Bilddateien in gültigen Ordnern:

Ordner: 20250221_Mini3Pro_Linde (82 Bilddateien)
  - DJI_0137.JPG
  - DJI_0138.JPG
  - DJI_0217.JPG
  - DJI_0218.JPG

Ordner: 20250228_Mini3Pro_Linde (144 Bilddateien)
  - DJI_0396.JPG
  - DJI_0397.JPG
  - DJI_0538.JPG
  - DJI_0539.JPG

Ordner: 20250305_Mini3Pro_Linde (124 Bilddateien)
  - DJI_0001.JPG
  - DJI_0002.JPG
  - DJI_0998.JPG
  - DJI_0999.JPG

Ordner: 20250403_Mini3Pro_Linde (424 Bilddateien)
  - DJI_0001.JPG
  - DJI_0002.JPG
  - DJI_0998.JPG
  - DJI_0999.JPG

Ordner: 20250508_Mini3Pro_Linde (551 Bilddateien)
  - DJI_0001.JPG
  - DJI_0002.JPG
  - DJI_0998.JPG
  - DJI_0999.JPG



In [8]:
import os
import re
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS
import statistics

# Basisverzeichnis
base_dir = r"G:\43_DJI_Mini3Pro"
valid_extensions = (".jpg", ".jpeg", ".JPG", ".JPEG")
pattern = re.compile(r"^\d{8}_Mini3Pro_Linde$")

# EXIF-Felder definieren
def get_exif_data(filepath):
    try:
        with Image.open(filepath) as img:
            exif_data = img._getexif()
            if not exif_data:
                return None
            labeled = {TAGS.get(k, k): v for k, v in exif_data.items()}
            return {
                "ISO": labeled.get("ISOSpeedRatings"),
                "FNumber": labeled.get("FNumber"),
                "ExposureTime": labeled.get("ExposureTime")
            }
    except Exception as e:
        print(f"Fehler bei Datei {filepath}: {e}")
        return None

# Ergebnisse speichern
summary_data = []

# Durch Ordner iterieren
for folder in sorted(os.listdir(base_dir)):
    if not pattern.match(folder):
        continue

    folder_path = os.path.join(base_dir, folder)
    image_files = sorted([
        f for f in os.listdir(folder_path)
        if f.endswith(valid_extensions)
    ])

    iso_list, f_list, exp_list = [], [], []

    for f in image_files:
        full_path = os.path.join(folder_path, f)
        exif = get_exif_data(full_path)
        if exif and all(exif.values()):
            iso_list.append(exif["ISO"])
            f_list.append(float(exif["FNumber"]))
            try:
                exp_val = exif["ExposureTime"]
                if isinstance(exp_val, tuple):  # z. B. (1, 4000)
                    exp = exp_val[0] / exp_val[1]
                else:
                    exp = float(exp_val)
                exp_list.append(exp)
            except:
                continue

    # Nur speichern, wenn Daten vorhanden
    if iso_list and f_list and exp_list:
        summary_data.append({
            "Ordner": folder,
            "ISO_min": min(iso_list),
            "ISO_max": max(iso_list),
            "ISO_mean": round(statistics.mean(iso_list), 1),
            "ISO_median": statistics.median(iso_list),
            "F_min": min(f_list),
            "F_max": max(f_list),
            "F_mean": round(statistics.mean(f_list), 2),
            "F_median": round(statistics.median(f_list), 2),
            "Exp_min": round(min(exp_list), 6),
            "Exp_max": round(max(exp_list), 6),
            "Exp_mean": round(statistics.mean(exp_list), 6),
            "Exp_median": round(statistics.median(exp_list), 6)
        })

# Als DataFrame anzeigen und speichern
output_dir = r"C:\_Data\BTh11\BTh11_Trieblaengenwachstum\15_Calendar_Vis\151_Output"

df_summary = pd.DataFrame(summary_data)
output_file = os.path.join(output_dir, "exif_auswertung_mini3pro_linde.csv")
df_summary.to_csv(output_file, index=False, sep=";")
print(f"\nEXIF-Auswertung gespeichert unter: {output_file}")



EXIF-Auswertung gespeichert unter: C:\_Data\BTh11\BTh11_Trieblaengenwachstum\15_Calendar_Vis\151_Output\exif_auswertung_mini3pro_linde.csv
